### Setup and import graph

In [1]:
######## setup variables ##########
gs_prefix="gs://anand-31-simple-files/customer-claims/customer-360"
#cluster_name="katana-internal31"
partitions = 2
prefix = gs_prefix
platform_prefix= prefix

In [2]:
import os                                                                        
import time                                                                      
import json
import pandas as pd
from timeit import default_timer as timer
from katana.remote import aio, analytics, import_data, trace

from katana import remote
from katana.remote import import_data

from katana.remote.analytics import connected_components, louvain_clustering

os.environ["KATANA_SERVER_ADDRESS"] = "host.docker.internal:8080"

pd.set_option('display.max_columns', None)
pd.set_option("max_colwidth", None)

In [3]:
rc=remote.Client()
rc.server_version

'0.2.2.dev+qa.5.55.bf870a.0c657d'

In [4]:

graph = rc.create_graph(
    num_partitions=partitions,
    name="customer-claims"
)

In [5]:
graph.graph_id

'EXG5KgDxpBQNLKGe5ECmQb9fvvKCLehEPhYWLBmqJQPr'

In [6]:
test_node_file = platform_prefix + "/42_NodeHeader.txt"
test_edge_file = platform_prefix + "/43_EdgeHeader.txt"
input_path=platform_prefix 


In [7]:
start = timer()
print("Importing graph from CSV files...")
import_data.csv(
    graph,
    input_node_path=test_node_file,
    input_edge_path=test_edge_file,
    input_dir=input_path,
#    have_headers=True,
    data_delimiter="|",
    schema_delimiter="|"
    #partition_policy="random-oec"
)
end = timer()
print(f"  import: {end - start:.1f} seconds") 

Importing graph from CSV files...


          0/? [?op/s]

  import: 98.0 seconds


In [8]:
%%time
graph.num_nodes()

          0/? [?op/s]

CPU times: user 188 ms, sys: 73.5 ms, total: 261 ms
Wall time: 8.24 s


2901419

In [ ]:
graph.num_edges()

## Investigate the neighbourhood of an Claimant with high payouts

In [9]:
%%time
#High number of payouts
query1="""
        MATCH (p:Claimant)<-[]-(py:Payment)
        RETURN count(py) as count, p.id order by count desc
        """
result = graph.query(query1, contextualize=True)
result.view()

          0/? [?op/s]

CPU times: user 2.38 s, sys: 616 ms, total: 2.99 s
Wall time: 23.1 s


<b> Let's have a look at the claimant neighborhood </b>

In [30]:
%%time
#Payments
query1="""
        MATCH (p:Claimant)<-[]-(py:Payment)
        where p.id = 'C_8900'
        RETURN p, py
        """
result = graph.query(query1, contextualize=True)
result.view()

          0/? [?op/s]

          0/? [?op/s]

CPU times: user 1.12 s, sys: 334 ms, total: 1.46 s
Wall time: 27.7 s


In [ ]:
%%time
#check Folks involved in common claim
query1="""
        MATCH (p:Claimant)-[:MADE]->(c:Claim)<-[:MADE]-(p2)
        where p.id = 'C_8900'
        RETURN p, c, p2
        """
result = graph.query(query1, contextualize=True)
result.view()

### Enriching the Topology: Duplicate Detection and Linking
We will pick claimants connected to C_8900 via common claims and look for duplicates

In [32]:
%%time
query1="""
        MATCH (p:Claimant)-[]-(c:Claim)<-[]-(p2:Claimant)
        where p.id = 'C_8900'
        RETURN distinct(p2) order by p2.firstname
        """
result = graph.query(query1)
result.view()

          0/? [?op/s]

CPU times: user 537 ms, sys: 154 ms, total: 690 ms
Wall time: 6.26 s


In [33]:
candidates = []
for e in result.iterrows():
    candidate = dict()
    candidate['firstname'] = e[1]['p2.firstname']
    candidate['lastname'] = e[1]['p2.lastname']
    candidate['id'] = e[1]['p2.id']
    candidate['address'] = e[1]['p2.address']
    candidates.append(candidate)

In [34]:
len(candidates)

77

In [35]:
import difflib
par1 = 'Daniel Williams'
par2 = 'Daniel William'
score = difflib.SequenceMatcher(None, par1, par2).ratio()
score

0.9655172413793104

In [36]:
def write_duplicates(graph, candidates):
    import csv
    import difflib
    print("Finding duplicates")
    import socket
    print(socket.gethostname())
    
    pos = int(socket.gethostname().replace('katana-compute-',''))
    lastIndex = len(candidates) - 1
    print('Last position of candidates to compare', lastIndex)
    duplicates=[]
    while pos < lastIndex:
        nameOfSource = candidates[pos]['firstname'] + candidates[pos]['lastname']
        addressOfSource = candidates[pos]['address']
        idOfSource = candidates[pos]['id']
        looper = 0
        while looper < lastIndex:
            idOfTarget = candidates[looper]['id']
            if idOfSource == idOfTarget:
                looper = looper + 1
                continue
            nameOfTarget = candidates[looper]['firstname'] + candidates[looper]['lastname']
            addressOfTarget = candidates[looper]['address']
            score = difflib.SequenceMatcher(None, nameOfSource, nameOfTarget).ratio()
            score_address = difflib.SequenceMatcher(None, addressOfSource, addressOfTarget).ratio()
            if(score > 0.8) and (score_address > 0.8):
                print('duplicates', idOfSource, idOfTarget)
                print('addressand score', addressOfSource, addressOfTarget, score_address)
                duplicate = dict()
                duplicate['idOfSource'] = idOfSource
                duplicate['idOfTarget'] = idOfTarget
                duplicates.append(duplicate)
            looper = looper + 1
        pos = pos + 5
    print('processing complete for node', socket.gethostname())
    print('last position compared', pos)
    
    outFileName = '/tmp/dups-1.csv'
    fields = ['idOfSource', 'idOfTarget']
    with open(outFileName, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fields, delimiter='|')
        writer.writeheader()
        if (len(duplicates) > 0): 
            writer.writerows(duplicates)
    print('finished writing duplicates')
    
#graph.run(lambda g: dist_count(g, 'argument1', 'argument2', 'argument3'))
#candidates = ['argument1', 'argument2', 'argument3','argument4', 'argument5']
graph.run(lambda g: write_duplicates(g, candidates))


          0/? [?op/s]

Finding duplicates
katana-compute-2
Last position of candidates to compare 76
duplicates C_312900 C_84200
addressand score 23222 Tammy Lane Smithvill US 93158 23222 Tammie Lane Smithville US 93158 0.9444444444444444
duplicates C_236100 C_16800
addressand score 36425 Jessica Alley East Alexa United States Minor Outlying Islands 29307 36425 Jess Alley East Alexa United States Minor Outlying Islands 29307 0.9790209790209791
processing complete for node katana-compute-2
last position compared 77
finished writing duplicates


In [ ]:
duplicates = []
duplicates.append(['C_312900','C_84200'])
duplicates.append(['C_236100','C_16800'])
duplicates.append(['C_106500','C_216400'])
len(duplicates)

In [9]:
#Mutation 1
query1="""
        MATCH (p1:Claimant),(p2:Claimant),(p3:Claimant),(p4:Claimant),(p5:Claimant),(p6:Claimant)
        where p1.id = '_d1_'
          and p2.id = '_d2_'
          and p3.id = '_d3_'
          and p4.id = '_d4_'
          and p5.id = '_d5_'
          and p6.id = '_d6_'
        with p1, p2, p3, p4, p5, p6
        create (p1)-[:isDuplicatedBy]->(p2)-[:isDuplicatedBy]->(p1),
               (p3)-[:isDuplicatedBy]->(p4)-[:isDuplicatedBy]->(p3),
               (p5)-[:isDuplicatedBy]->(p6)-[:isDuplicatedBy]->(p5)
        RETURN p1, p2, p3, p4, p5, p6
        """
query1 = query1.replace('_d1_', 'C_312900')
query1 = query1.replace('_d2_', 'C_84200')
query1 = query1.replace('_d3_', 'C_236100')
query1 = query1.replace('_d4_', 'C_16800')
query1 = query1.replace('_d5_', 'C_106500')
query1 = query1.replace('_d6_', 'C_216400')

print(query1)
result = graph.query(query1)


        MATCH (p1:Claimant),(p2:Claimant),(p3:Claimant),(p4:Claimant),(p5:Claimant),(p6:Claimant)
        where p1.id = 'C_312900'
          and p2.id = 'C_84200'
          and p3.id = 'C_236100'
          and p4.id = 'C_16800'
          and p5.id = 'C_106500'
          and p6.id = 'C_216400'
        with p1, p2, p3, p4, p5, p6
        create (p1)-[:isDuplicatedBy]->(p2)-[:isDuplicatedBy]->(p1),
               (p3)-[:isDuplicatedBy]->(p4)-[:isDuplicatedBy]->(p3),
               (p5)-[:isDuplicatedBy]->(p6)-[:isDuplicatedBy]->(p5)
        RETURN p1, p2, p3, p4, p5, p6
        


          0/? [?op/s]

In [ ]:
%%time
#check duplicate objects
query1="""
        MATCH (p1:Claimant)-[:isDuplicatedBy]->(p2:Claimant)
        RETURN p1, p2
        """
result = graph.query(query1, contextualize=True)
result.view()

## Revisit the neighborhood and check for a fraudulent patterns

In [ ]:
%%time
#check Folks involved in common claim
query1="""
        MATCH (p:Claimant)-[:MADE]->(c:Claim)<-[e:MADE]-(p2)
        where p.id = 'C_8900'
        with p, c, p2
        MATCH (p:Claimant)-[:MADE]->(c:Claim)<-[e1:MADE]-(p2)
        where e1.ROLE = 'witness'
        RETURN p, c, p2
        """
result = graph.query(query1, contextualize=True)
result.view()

## Project the claimant-claimant relationship, and then revisit the neighborhood of C_8900

In [10]:
%%time
query1="""
        MATCH (p1:Claimant)-[]-(c:Claim)-[]-(p2:Claimant)
        with p2,p1, count(c) as common_claim_count 
        create (p1)-[:common_claim{count: common_claim_count}]->(p2)-[:common_claim{count: common_claim_count}]->(p1)
        """
result = graph.query(query1)
result.view()

          0/? [?op/s]

CPU times: user 1.49 s, sys: 304 ms, total: 1.79 s
Wall time: 23.5 s


## Pagerank and Louvain Algorithms

In [11]:
from katana.remote.analytics import pagerank
pagerank(graph, 'pg2')

          0/? [?op/s]

In [13]:
gp2 = graph.project(node_types = ['Claimant'], edge_types = ['common_claim', 'isDuplicatedBy'])
louvain_clustering(gp2,'lv3',  is_symmetric=True)

          0/? [?op/s]

In [39]:
%%time
#check top clusters
query1="""
        MATCH (p:Claimant)
        with avg(toFloat(p.pg2))as avg_pr, count(p) as count, p.lv3 as lv
        where count > 1
        return avg_pr, count, lv order by avg_pr desc limit 10
        """
result = graph.query(query1)
result.view()

          0/? [?op/s]

CPU times: user 378 ms, sys: 108 ms, total: 486 ms
Wall time: 18.9 s


In [38]:
%%time
#Expand 3 cluster 
query1="""
        MATCH (p)
        where tointeger(p.lv3) = 125210
        with p
        match (p)-[]-(q)
        where not ( q:Claimant) 
        return q, p order by q.amount desc limit 100
        """
result = graph.query(query1, contextualize = True)
result.view()

          0/? [?op/s]

          0/? [?op/s]

CPU times: user 1.68 s, sys: 482 ms, total: 2.16 s
Wall time: 26.9 s


In [37]:
%%time
#Zoom in on the top one
query1="""
        MATCH (p)
        where tointeger(p.lv3) = 125210
        with p
        match (p)-[]-(q)
        where not (q:Claim OR q:Claimant) 
        return p, q
        """
result = graph.query(query1, contextualize = True)
result.view()

          0/? [?op/s]

          0/? [?op/s]

CPU times: user 1.27 s, sys: 449 ms, total: 1.72 s
Wall time: 34.3 s


### Appendix

In [36]:
%%time
#contrast average payouts
query1="""
        MATCH (p:Claimant)
        with avg(toFloat(p.pg2))as avg_pr, count(p) as count, p.lv3 as lv
        where count > 1
        with avg_pr, count, lv order by avg_pr desc limit 20 
        match (x:Claimant)<-[:WAS_PAID]-(py:Payment)
        where tointeger(x.lv3) = lv
        return avg_pr, x.lv3, avg(toFloat(py)) as average_payout order by avg_pr desc limit 10
        """
result = graph.query(query1)
result.view()

          0/? [?op/s]

CPU times: user 250 ms, sys: 74.8 ms, total: 325 ms
Wall time: 20.3 s


In [12]:
%%time
#contrast avg payment - 2
query1="""
        MATCH (p:Claimant)
        where exists(p.pg2)
        with p order by p.pg2 desc
        MATCH (c:Claim)<-[:MADE]-(p)<-[:WAS_PAID]-(py:Payment)
        return avg(py.amount) as average_payments_amount, p.id order by average_payments_amount desc limit 10
        """
result = graph.query(query1)
result.view()

          0/? [?op/s]

CPU times: user 326 ms, sys: 101 ms, total: 427 ms
Wall time: 11.7 s


In [27]:
%%time
#check top nodes
query1="""
        MATCH (p:Claimant)
        where exists(p.pg2)
        RETURN p order by p.pg2 desc  limit 5
        """
result = graph.query(query1)
result.view()

          0/? [?op/s]

CPU times: user 748 ms, sys: 187 ms, total: 935 ms
Wall time: 10.4 s


In [25]:
%%time
#check top nodes
query1="""
        MATCH (att:Attorney)
        where att.id = 'X_4853'
        RETURN att
        """
result = graph.query(query1)
result.view()

          0/? [?op/s]

CPU times: user 657 ms, sys: 131 ms, total: 788 ms
Wall time: 12.5 s


In [25]:
%%time
#Get suspicious claims
query1="""
        MATCH (p1:Claimant)-[:isDuplicatedBy]-(p2:Claimant)-[MADE]-(cl:Claim)-[:REPRESENTS]->(att:Attorney)
        where att.id = 'X_4853'
        RETURN p1,p2,cl,att
        """
result = graph.query(query1, contextualize=True)
result.view()

          0/? [?op/s]

          0/? [?op/s]

CPU times: user 3.1 s, sys: 550 ms, total: 3.65 s
Wall time: 38.5 s


          0/? [?op/s]

'8SsgiQsdCKsjagFxciuAVScd2d6wRX9cBT5tdeXnGs3N'

## Page rank on projected graph

In [18]:
gp4 = graph.project(node_types = ['Claimant'], edge_types = ['common_claim', 'isDuplicatedBy'])
from katana.remote.analytics import pagerank
pagerank(gp4, 'pgp')

          0/? [?op/s]

In [ ]:
%%time
#check top nodes
query1="""
        MATCH (p)
        where exists(p.pgp)
        RETURN p order by p.pgp desc  limit 5
        """
result = graph.query(query1)
result.view()

In [17]:
%%time
#check top nodes
query1="""
        MATCH (p)-[]-(q:Claim)
        where p.id = 'C_363900'
        RETURN p,q
        """
result = graph.query(query1, contextualize = True)
result.view()

          0/? [?op/s]

          0/? [?op/s]

CPU times: user 3.5 s, sys: 1.07 s, total: 4.56 s
Wall time: 25.5 s


In [31]:
%%time
#check transaction objects
query1="""
        MATCH (p:Claimant)
        where exists(p.pgp)
        with p order by p.pgp desc  limit 20
        Match(p:Claimant)-[]-(p1:Claimant)-[:isDuplicatedBy]-(p2:Claimant)
        with distinct(p)
        match (p:Claimant)-[:MADE]->(q:Claim)
        return p,q 
        """
result = graph.query(query1, contextualize=True)
result.view()

          0/? [?op/s]

          0/? [?op/s]

CPU times: user 1.87 s, sys: 519 ms, total: 2.39 s
Wall time: 26.1 s


In [32]:
%%time
#check transaction objects
query1="""
        MATCH (p:Claimant)
        where exists(p.pgp)
        with p order by p.pgp desc  limit 20
        Match(p:Claimant)-[]-(p1:Claimant)-[:isDuplicatedBy]-(p2:Claimant)
        with distinct(p)
        match (p:Claimant)-[:common_claim]->(q:Claimant)
        return p,q 
        """
result = graph.query(query1, contextualize=True)
result.view()

          0/? [?op/s]

          0/? [?op/s]

CPU times: user 2.77 s, sys: 961 ms, total: 3.73 s
Wall time: 24.4 s


### Louvain
run louvain + pagerank
produce a query with louvain cluster with average pagerank, sort by page rank desc, asc


In [13]:
gp2 = graph.project(node_types = ['Claimant'], edge_types = ['common_claim', 'isDuplicatedBy'])
louvain_clustering(gp2,'lv3',  is_symmetric=True)

          0/? [?op/s]

In [ ]:
%%time
#check transaction objects
query1="""
        MATCH (c:Claim)<-[:MADE]-(p:Claimant)<-[:WAS_PAID]-(py:Payment)
        return avg(py.amount) as average_payments_amount
        """
result = graph.query(query1)
result.view()


In [36]:
%%time
#check transaction objects
query1="""
        MATCH (p:Claimant)-[]-(q)
        where p.id = 'C_8900'
        RETURN p, q
        """
result = graph.query(query1, contextualize=True)
result.view()

          0/? [?op/s]

          0/? [?op/s]

CPU times: user 1.18 s, sys: 322 ms, total: 1.5 s
Wall time: 27 s


In [43]:
%%time
#check transaction objects
query1="""
        MATCH (p:Claimant)
        return sum(p.pg2 )as sum_pr, count(p) as count, p.lv3 as lv order by sum_pr desc limit 10
        """
result = graph.query(query1)
result.view()

          0/? [?op/s]

CPU times: user 197 ms, sys: 72.8 ms, total: 270 ms
Wall time: 12.4 s


In [44]:
%%time
#check transaction objects
query1="""
        MATCH (p:Claimant)-[]-(b)
        where tointeger(p.lv3) = 2373
        RETURN p,b
        """
result = graph.query(query1, contextualize=True)
result.view()

          0/? [?op/s]

          0/? [?op/s]

CPU times: user 2.17 s, sys: 458 ms, total: 2.63 s
Wall time: 36.6 s


## Appendix

In [12]:
graph.graph_id

'EtUsiRFnqzyXf8iPvMzbK47fMok5JuGKvbvPJ41QyuEe'

In [16]:
%%time
#check transaction objects
query1="""
        MATCH (c:Claim)<-[:MADE]-(p:Claimant)<-[:WAS_PAID]-(py:Payment)
        return avg(py.amount) as average_payments_amount
        """
result = graph.query(query1)
result.view()


          0/? [?op/s]

CPU times: user 240 ms, sys: 63.3 ms, total: 304 ms
Wall time: 10.1 s


In [17]:
# add a query that produces page rank for average payment amount

          0/? [?op/s]

CPU times: user 377 ms, sys: 95.8 ms, total: 473 ms
Wall time: 10.3 s


## Claimants like C_231800 can be further investigated as they have nearly double the average payment going out

In [15]:
%%time
#check transaction objects
query1="""
        MATCH (p:Claimant)
        where p.id = 'C_231800'
        return p
        """
result = graph.query(query1)
result.view()

          0/? [?op/s]

CPU times: user 464 ms, sys: 164 ms, total: 629 ms
Wall time: 10.4 s


In [16]:
%%time
#Claimants with high number of payments
query1="""
        MATCH (p:Claimant)
        where exists(p.pg2)
        with p order by p.pg2 desc
        MATCH (c:Claim)<-[:MADE]-(p)<-[:WAS_PAID]-(py:Payment)
        return count(py) as total_payments_made, p.id order by total_payments_made desc limit 10
        """
result = graph.query(query1)
result.view()

          0/? [?op/s]

CPU times: user 369 ms, sys: 83.7 ms, total: 453 ms
Wall time: 5.29 s


### Other operations

In [39]:
from katana.remote.analytics import connected_components
connected_components(gp, 'cc2')

          0/? [?op/s]

In [40]:
%%time
#check transaction objects
query1="""
        MATCH (p:Claimant)
        where p.id = 'C_8900'
        RETURN p
        """
result = graph.query(query1)
result.view()

          0/? [?op/s]

CPU times: user 755 ms, sys: 225 ms, total: 980 ms
Wall time: 14.8 s


In [41]:
%%time
#check transaction objects
query1="""
        MATCH (p1)-[]-(p2)
        where tointeger(p1.cc2) = 0
        return p1, p2
        """
result = graph.query(query1, contextualize = True)
result.view()

          0/? [?op/s]

CPU times: user 12.6 s, sys: 2.01 s, total: 14.6 s
Wall time: 53.1 s


In [45]:

%%time
#check transaction objects
query1="""
        CALL graph.schema() return *;
        """
result = graph.query(query1)
result.view()

          0/? [?op/s]

CPU times: user 621 ms, sys: 227 ms, total: 848 ms
Wall time: 1min 28s


In [46]:
result.to_csv()

',neighbor,nodeType,properties\n0,,[Agent],"dob,email,firstname,id,lastname,pg2,ssn"\n1,([Location]::[IS_LOCATED_IN]),[Agent],\n2,,[Attorney],"dob,email,firstname,id,lastname,pg2,ssn"\n3,([Location]::[IS_LOCATED_IN]),[Attorney],\n4,,[Claim],"amount,attorney_id,capital_gains,capital_loss,claim_date,claimant_id,id,pg2,policy_id,witnesses"\n5,([Attorney]::[REPRESENTS]),[Claim],\n6,,[Claimant],"address,dob,email,firstname,id,lastname,lv3,pg2,ssn"\n7,([Claim]::[IS_LOCATED_IN]),[Claimant],\n8,([Claim]::[MADE]),[Claimant],ROLE\n9,([Claim]::[common_claim]),[Claimant],count\n10,([Claimant]::[MADE]),[Claimant],ROLE\n11,([Claimant]::[common_claim]),[Claimant],count\n12,([Claimant]::[isDuplicatedBy]),[Claimant],\n13,([Location]::[IS_LOCATED_IN]),[Claimant],\n14,([Location]::[common_claim]),[Claimant],count\n15,,[Location],"city,country,id,pg2,postalcode_plus4,state_abbr,street_address"\n16,,[Mbr],"amount,authorities_contacted,claim_id,id,incident_city,incident_date,incident_severity,incident_state